In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['final_merged.tsv']


In [2]:
data = pd.read_csv('../input/final_merged.tsv', sep='\t')

Read the final feature processed dataset

In [3]:
data.head()

,User_Id,Prod_Id,Date_x,Review,Rating,Label,Product_Name,Avg_Prod_Rating,Avg_user_rating,Review_Len,user_total_reviews,corpus,compound,neg,neu,pos,number_Cap_Words,number_digit_Words,noun_count
0,923,0,08-12-2014,The food at snack is a selection of popular Gr...,3,-1,Snack,4.009524,4.435897,215,39,food snack select popular greek dish appet tra...,0.6486,0.061,0.693,0.247,0,0,3
1,923,19,14-01-2014,The restaurant is on the ground floor of a typ...,5,-1,Palo Santo,4.037152,4.435897,513,39,littl place soho wonder lamb sandwich glass wi...,-0.1280,0.189,0.673,0.138,0,0,9
2,923,40,30-05-2014,Really nice mousaka and lovely décor inside. A...,4,-1,Pylos,4.312869,4.435897,231,39,order lunch snack last friday time noth miss f...,0.7717,0.067,0.628,0.305,0,1,4
3,923,63,13-11-2014,I really enjoyed brunch at Jane. The ambiance ...,4,-1,Jane,3.937181,4.435897,169,39,beauti quaint littl restaur pretti street stro...,0.8910,0.042,0.712,0.246,0,0,2
4,923,79,30-03-2014,We ate at the Blue Ribbon with colleagues. The...,3,-1,Blue Ribbon Brasserie,4.280000,4.435897,255,39,snack great place casual sit lunch especi cold...,0.9769,0.000,0.661,0.339,0,0,3


**Identifying the most common words for Fake and True reviews**

In [4]:
positive = data[data['Label']== 1]
true_word_list = []
for i in range(0,positive['corpus'].count()):
   true_word_temp = positive['corpus'].iloc[i].split()
   true_word_list.append(true_word_temp)
from itertools import chain
list1 = list(chain.from_iterable(true_word_list))
from collections import Counter
true_mc = Counter(list1).most_common()
 
 
fake = data[data['Label'] == -1]
fake_word_list= fake['corpus'].str.split(expand=True).stack()
from collections import Counter
fake_mc = Counter(fake_word_list).most_common()
fake_mc
 
df_true = pd.DataFrame(true_mc)
df_false = pd.DataFrame(fake_mc)
df_false1 = df_false[0:23]
df_true1 = df_true[0:23]
df_common = list(set(df_true1.iloc[:,0]) & set(df_false1.iloc[:,0]))
df_common

['also',
 'place',
 'would',
 'wait',
 'food',
 'order',
 'love',
 'great',
 'restaur',
 'get',
 'servic',
 'one',
 'good',
 'realli',
 'tri',
 'back',
 'best',
 'like',
 'delici',
 'go',
 'come',
 'time']

Dropping the columns that are not required by the model

In [5]:
data = data.dropna()
final_data_for_train =  data.drop(columns=['User_Id','Prod_Id','Date_x','Review ','Product_Name','compound'])

Vectorization and OneHotEncoding

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
 
#Split
X_train, X_test, y_train, y_test = train_test_split(final_data_for_train, data['Label'], test_size=0.25, random_state=101)
#Vectorize
vectorizer = CountVectorizer(stop_words= df_common, ngram_range=(3,3), max_features= 15000)

In [7]:
vectorizer.fit(X_train['corpus'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=15000, min_df=1,
        ngram_range=(3, 3), preprocessor=None,
        stop_words=['also', 'place', 'would', 'wait', 'food', 'order', 'love', 'great', 'restaur', 'get', 'servic', 'one', 'good', 'realli', 'tri', 'back', 'best', 'like', 'delici', 'go', 'come', 'time'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [8]:
X_train_vector = vectorizer.transform(X_train['corpus'])
X_test_vector  = vectorizer.transform(X_test['corpus'])

In [9]:
import scipy as sp
final_train = sp.sparse.hstack((X_train_vector,X_train[['Rating','Avg_Prod_Rating','Avg_user_rating','Review_Len','user_total_reviews','neu','neg','pos','number_Cap_Words','number_digit_Words','noun_count']].values),format='csr')
final_test = sp.sparse.hstack((X_test_vector,X_test[['Rating','Avg_Prod_Rating','Avg_user_rating','Review_Len','user_total_reviews','neu','neg','pos','number_Cap_Words','number_digit_Words','noun_count']].values),format='csr')

Random over sampling technique is used to handle unbalanced dataset

In [10]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(final_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

Using TensorFlow backend.


[(-1, 265849), (1, 265849)]


In [11]:
print(X_resampled.shape)

(531698, 15011)


Using Multinomial Naive bayes model as the data is discrete

In [13]:
from sklearn.naive_bayes import MultinomialNB 
mnb = MultinomialNB().fit(X_resampled,y_resampled)


In [14]:
mnb_pred=mnb.predict(final_test)

Metrics for evaluating the model

In [15]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score

In [16]:
print(classification_report(mnb_pred, y_test))
print(roc_auc_score(mnb_pred, y_test))
print(confusion_matrix(mnb_pred, y_test))

              precision    recall  f1-score   support

          -1       0.86      0.16      0.27     55389
           1       0.48      0.97      0.64     43453

   micro avg       0.52      0.52      0.52     98842
   macro avg       0.67      0.56      0.46     98842
weighted avg       0.69      0.52      0.43     98842

0.5649108170681592
[[ 8988 46401]
 [ 1410 42043]]


In [17]:
print(accuracy_score(mnb_pred, y_test))

0.5162886222456041


SMOTE over sampling technique  used to balance the unbalanced dataset

In [18]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(final_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(-1, 265849), (1, 265849)]


Using Multinomial Naive bayes model as the data is discrete

In [19]:
from sklearn.naive_bayes import MultinomialNB 
mnb = MultinomialNB().fit(X_resampled,y_resampled)


In [20]:
mnb_pred=mnb.predict(final_test)

Metrics for evaluating the model

In [21]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score

In [22]:
x=classification_report(mnb_pred, y_test)
print(classification_report(mnb_pred, y_test))
print(roc_auc_score(mnb_pred, y_test))
print(confusion_matrix(mnb_pred, y_test))

              precision    recall  f1-score   support

          -1       0.88      0.16      0.27     57574
           1       0.45      0.97      0.62     41268

   micro avg       0.50      0.50      0.50     98842
   macro avg       0.67      0.56      0.44     98842
weighted avg       0.70      0.50      0.41     98842

0.5646334830234907
[[ 9164 48410]
 [ 1234 40034]]


In [23]:
print(accuracy_score(mnb_pred, y_test))

0.49774387406163373
